In [19]:
import numpy as np
import pandas as pd
import random as python_random
import shutil
import os
from os import listdir
import datetime
from pathlib import Path
from PIL import UnidentifiedImageError
from PIL import Image
from multiprocessing import cpu_count
from joblib import Parallel, delayed
from collections import Counter
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, mean_absolute_error

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# for reproducibility purposes
SEED = 123
tf.random.set_seed(SEED)

# load tensorboard extension
%reload_ext tensorboard
# specify the log directory where the tensorboard logs will be written
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 25096), started 6 days, 21:40:30 ago. (Use '!kill 25096' to kill it.)

In [20]:
#commonly used batch size
batch_size = 50

# Used to augment images, the below augmentations are used to augment the images and output as numeric data for machine learning
train_img_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.1,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255,
    preprocessing_function=None,
    data_format=None,
    # splitting 20% of the training data into validation data
    validation_split=0.2,
    dtype=None
)
# gets all of the images and labels from the training folder and applies the above augmentations for the training subset of data
flowfromframeTrain = train_img_gen.flow_from_directory(
    directory='./input/train',
    target_size=(224, 224),
    color_mode='rgb',
    classes=None,
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=True,
    seed=SEED,
    save_to_dir=None,
    save_prefix='',
    save_format='',
    follow_links=False,
    subset='training',
    interpolation='nearest'
)
    
flowfromframeVal = train_img_gen.flow_from_directory(
    directory='./input/train',
    target_size=(224, 224),
    color_mode='rgb',
    classes=None,
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=True,
    seed=SEED,
    save_to_dir=None,
    save_prefix='',
    save_format='',
    follow_links=False,
    subset='validation',
    interpolation='nearest'
)

test_img_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=1./255,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None
)

flowfromframeTest = test_img_gen.flow_from_directory(
    directory='./input/test',
    target_size=(224, 224),
    color_mode='rgb',
    classes=None,
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=True,
    seed=SEED,
    save_to_dir=None,
    save_prefix='',
    save_format='',
    follow_links=False,
    interpolation='nearest'
)

Found 7632 images belonging to 16 classes.
Found 1900 images belonging to 16 classes.
Found 814 images belonging to 16 classes.


In [21]:
# VGG16 model is initialized here
# model = tf.keras.applications.vgg16.VGG16(
#     include_top=True,
#     weights=None,
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=22,
#     classifier_activation='softmax'
# )
input_shape=(224, 224, 3)
model = tf.keras.Sequential([
  
  layers.Conv2D(16, 3, padding='same', activation='relu',input_shape=input_shape),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(16, activation='softmax')
])

learn_rate = (0.001)
optimizer = tf.keras.optimizers.RMSprop(learning_rate=learn_rate)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model_Callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3),
]

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='./data/model',
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode="auto",
    save_freq="epoch",
    options=None,
    initial_value_threshold=None,
)
model_Callbacks.append(model_checkpoint)

reduceLRCallback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=2,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0.0001,
)
model_Callbacks.append(reduceLRCallback)

numStepsPerEpoch = flowfromframeTrain.total_batches_seen
model.fit(
    x=flowfromframeTrain,
    y=None,
    batch_size=batch_size,
    epochs=10,
    verbose=1,
    callbacks=model_Callbacks,
    validation_split=0.0,
    validation_data=flowfromframeVal,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=100,
    validation_steps=50,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)




Epoch 1/10
10/10 [==============================] - 12s 1s/step - loss: 3.8511 - accuracy: 0.0720 - val_loss: 2.7337 - val_accuracy: 0.1240
INFO:tensorflow:Assets written to: ./data\model\assets
Epoch 2/10
10/10 [==============================] - 12s 1s/step - loss: 2.6742 - accuracy: 0.1460 - val_loss: 2.7012 - val_accuracy: 0.1160
INFO:tensorflow:Assets written to: ./data\model\assets
Epoch 3/10
10/10 [==============================] - 11s 1s/step - loss: 2.6418 - accuracy: 0.1660 - val_loss: 2.4900 - val_accuracy: 0.2640
INFO:tensorflow:Assets written to: ./data\model\assets
Epoch 4/10
10/10 [==============================] - 11s 1s/step - loss: 2.5509 - accuracy: 0.2080 - val_loss: 2.4392 - val_accuracy: 0.2400
INFO:tensorflow:Assets written to: ./data\model\assets
Epoch 5/10
10/10 [==============================] - 10s 1s/step - loss: 2.4419 - accuracy: 0.2160 - val_loss: 2.4020 - val_accuracy: 0.2080
INFO:tensorflow:Assets written to: ./data\model\assets
Epoch 6/10
10/10 [=======

In [28]:
scores = model.evaluate(flowfromframeTest)

print(scores)

predictions = model.predict(flowfromframeTest)

test_labels = flowfromframeTest.labels

y_pred = np.argmax(predictions,axis=-1)

for i in y_pred:
    print(i)

print(classification_report(test_labels, y_pred))

17/17 [==============================] - 3s 160ms/step - loss: 2.2730 - accuracy: 0.2850
[2.273045301437378, 0.28501227498054504]
6
5
13
8
8
10
10
0
8
13
8
5
8
0
5
5
6
10
8
5
8
2
10
5
15
7
4
11
15
4
11
12
12
7
8
8
10
8
13
10
5
0
4
11
8
13
5
4
13
6
6
4
5
4
6
7
8
8
2
8
13
10
8
8
5
4
8
4
5
6
8
9
5
5
8
8
0
8
5
4
10
8
13
15
6
0
5
8
0
8
4
14
7
5
8
5
10
7
10
15
7
6
0
0
7
5
8
5
5
13
4
11
7
3
0
5
4
8
5
7
10
0
0
8
10
11
0
12
8
13
12
8
0
5
15
10
8
10
8
6
5
5
8
8
7
5
5
5
7
5
12
13
8
5
7
5
6
5
11
8
10
0
0
6
1
8
13
5
5
6
8
11
11
0
0
11
12
2
5
0
8
10
15
8
10
11
0
0
0
4
8
8
8
8
8
5
4
0
8
5
3
6
5
2
10
8
5
8
8
5
0
10
8
8
6
0
8
5
12
5
4
5
10
6
6
8
4
4
8
10
0
4
8
5
4
11
11
10
5
11
10
7
7
0
8
8
1
11
5
10
8
8
0
8
6
11
12
10
4
5
10
8
8
11
8
0
8
13
5
7
8
10
7
8
13
13
6
4
8
11
15
8
8
8
11
4
12
5
10
8
10
4
8
6
8
8
10
15
4
8
0
11
5
8
8
8
12
0
2
6
8
5
5
0
15
5
8
0
5
5
8
11
0
6
13
10
13
7
8
6
0
5
5
4
4
5
13
6
10
0
0
11
4
15
0
8
10
10
8
8
8
8
10
8
13
8
10
5
8
13
0
8
6
8
5
7
5
10
2
8
11
8
8
5
0
10
5
11
13
0
15
5
2
8